# 차등 발현 및 클러스터 주석  

## 소개

이제 세포를 클러스터로 할당했으므로 각 클러스터가 데이터셋의 다른 세포와 다른 점을 이해하거나, 이 데이터셋에 대해 이전에 수행된 것처럼 세포 유형에 따라 클러스터에 주석을 달고 싶습니다.  

이 작업에는 몇 가지 접근 방식이 있습니다:  

* 관심 있는 세포 유형에 대한 마커 유전자의 상향 조절 찾기(데이터셋의 나머지 부분과 비교)  
* 그룹 간의 전체 유전자 발현 프로파일 비교  
* 관심 있는 세포를 세포 유형 발현 프로파일 데이터베이스와 비교하여 클러스터링과 주석을 결합하는 자동화된 방법 사용  

자동화된 방법은 유망한 발전이지만 아직 신중한 인간 큐레이션을 대체할 수는 없습니다. 

잘 정의된 세포 유형의 경우, 마커 유전자가 관심 있는 세포 유형과 데이터셋의 나머지 부분 간에 발현에 큰 차이를 보일 것으로 예상되므로 간단한 방법을 사용할 수 있습니다. 이 워크숍에서는 다른 접근 방식에 광범위하게 적용할 수 있는 직관을 구축하면서 이 접근 방식에 중점을 둘 것입니다.

## 데이터 로드

### 중요 참고! 차등 발현의 경우 `adata.raw`에 저장된 _원시_ 값을 사용해야 합니다.

차등 발현을 통해 각 그룹의 발현 중심과 확산을 모두 고려하고자 합니다. 값을 정규화할 때 각 유전자의 세포 간 분포를 0을 중심으로 하고 분산 1로 스케일링하도록 표준화했음을 상기하십시오. 따라서 차등 발현을 계산할 때 원시 값(QC 후, 정규화 전)을 사용해야 합니다. 이 값들은 이전에 `adata.raw`에 저장했습니다. 

In [1]:
import scanpy as sc
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

adata = sc.read('../data/brain_clusters.h5ad')
raw = pd.DataFrame(data=adata.raw.X, index=adata.raw.obs_names, columns=adata.raw.var_names)

## 분포 비교

차등 발현 알고리즘은 한 그룹의 유전자 발현 분포를 다른 그룹과 비교하는 다양한 접근 방식을 나타냅니다. 벌크 RNA-seq와 달리 단일 세포 실험에서는 비교하는 각 그룹에 대해 많은 수의 샘플(즉, 세포)이 있습니다. 따라서 벌크 RNASeq의 표준인 평균 발현 추정치만 비교하는 대신 각 그룹의 전체 발현 값 분포를 활용하여 그룹 간의 차이를 식별할 수 있습니다. 

### 직관 구축

몇 가지 직관을 구축하기 위해 몇 가지 기초 통계를 다시 살펴보겠습니다. 

이 예에서는 관심 있는 클러스터가 있으며, 이들이 성상세포인지 확인하려고 합니다. 클러스터에서 마커 유전자의 발현 분포를 데이터셋의 나머지 부분과 비교하여 플로팅하는 것으로 시작하겠습니다.

In [3]:
astrocyte_marker = 'Gja1' # 관심 유전자 정의
# housekeeping = 'Chmp2a' 

cluster2 = raw[adata.obs['louvain'] == '2'] # 마스크를 사용하여 데이터셋을 클러스터 2에 할당된 세포로 부분 집합화합니다.
not_cluster2 = raw[adata.obs['louvain'] != '2'] # 다른 모든 세포

In [31]:
sc.tl.rank_genes_groups(adata, groupby='cell_ontology_class', use_raw=True, 
                        method='t-test_overestim_var', n_genes=10) # compute differential expression
sc.pl.rank_genes_groups_tracksplot(adata, groupby='cell_ontology_class') # plot the result

### 연습

이 과정을 `louvain` 클러스터 할당에 대해 반복합니다. 이들이 얼마나 밀접하게 겹치는지 확인하십시오. 이것이 클러스터 레이블에 대한 Rand Index 평가 결과와 일치합니까?

<details><summary><h3>해결책</h3></summary>
<code style=display:block;white-space:pre-wrap>sc.tl.rank_genes_groups(adata, groupby='louvain', use_raw=True, 
                        method='t-test_overestim_var', n_genes=10) # 차등 발현 계산
sc.pl.rank_genes_groups_tracksplot(adata, groupby='louvain') # 결과 플로팅</code>

## "알려진" 마커 유전자와 비교  

또한 이러한 데이터 기반 마커 유전자를 현장에서 세포 유형의 고전적인 지표로 간주되는 것과 비교할 수 있습니다.  

In [20]:
sc.tl.rank_genes_groups(adata, groupby='cell_ontology_class', use_raw=True, 
                        method='t-test_overestim_var', n_genes=10) # compute differential expression
sc.pl.rank_genes_groups_tracksplot(adata, groupby='cell_ontology_class') # plot the result

### 연습

이 과정을 `louvain` 클러스터 할당에 대해 반복합니다. 이들이 얼마나 밀접하게 겹치는지 확인하십시오. 이것이 클러스터 레이블에 대한 Rand Index 평가 결과와 일치합니까?

<details><summary><h3>해결책</h3></summary>
<code style=display:block;white-space:pre-wrap>sc.tl.rank_genes_groups(adata, groupby='louvain', use_raw=True, 
                        method='t-test_overestim_var', n_genes=10) # 차등 발현 계산
sc.pl.rank_genes_groups_tracksplot(adata, groupby='louvain') # 결과 플로팅</code>

## "알려진" 마커 유전자와 비교  

또한 이러한 데이터 기반 마커 유전자를 현장에서 세포 유형의 고전적인 지표로 간주되는 것과 비교할 수 있습니다.  

In [20]:
marker_genes = {
'astrocyte': ['Aldh1l1', 'Slc1a3', 'Aqp4'], 
'oligodendrocyte': ['Mog','Mag'],
'oligodendrocyte precursor cell': ['Pdgfra','Susd5','Cspg4'],
'endothelial cell': ['Pecam1','Cldn5','Slco1c1','Ocln'],
'Bergmann glial cell': ['Gdf10','Vim','Nbl1','A2m'],
'excitatory neuron': ['Slc17a7','Neurod6','Mab21l1'],
'inhibitory neuron': ['Gad1','Reln','Calb1'],
'brain pericyte': ['Des','Mcam','Pdgfrb']
}

sc.pl.matrixplot(adata, marker_genes, groupby='louvain', use_raw=False)

GridSpec(2, 3, height_ratios=[0.5, 10], width_ratios=[8.0, 0, 0.2])

### 연습 

생각해보기 - 짝과 토론하기 - 공유하기

1. 이 플롯을 원시 값을 사용하여 다시 만들어보십시오. 결과에 대해 어떻게 생각하십니까?  
2. 사전에 주석이 달린 세포 유형이 없는 경우, 이러한 결과를 사용하여 세포를 세포 유형에 예비적으로 할당하려면 어떻게 하시겠습니까? 다음 단계는 무엇입니까?  
3. 이 플롯을 다시 만들어 이러한 마커 유전자가 `cell_ontology_class`의 사전 주석이 달린 세포 유형과 어떻게 겹치는지 검사해보십시오. 이 레이블에 동의하십니까?  

In [ ]:
help(sc.pl.matrixplot)